In [ ]:
%pip install dash

In [3]:
# USE DASH TO BUILD GUI
from dash import Dash, html, dcc, callback, Output, Input, ctx
import plotly.graph_objects as go
import plotly.express as px
# STANDARD IMPORTS
import pandas as pd
import os

In [ ]:
def generateField(trans:dict,medium:dict,field:dict,iscomplete=0):
    """
    INPUT ARG
        trans == dictionary of transducer properties
            "freq" == [Hz] Transmit Frequency of transducer
            "radius" == [m] Radius of transducer probe
            "focus" == [m] Focal point in space 
            "initPressure" == [Pa] Initial Pressure output by transducer

        medium == [dict] dictionary of medium properties
            "speed" == [m/s] Speed of Sound
            "density" == [kg/m^3] Density 
            "absCoeff" == [Np/(m*MHz^2)] Absorption Coefficient

        field == calls 2D field properties
            "numAxialStep" == [int] Number of Axial Steps
            "numRadialStep" == [int] Number of Radial Steps

    OUTPUT ARG
        pressure_field == [2D list][p/p0] Resulting Rayleigh Integral Pressure Field
        z_axis == [1D list][m] Axial axis of Pressure Field
        r_axis == [1D list][m] Radial axis of Pressure Field 

    """
    import numpy as np
    
    # Edit and Transform Transducer Properties
    d = trans["focus"] # Reallocate variable
    k = 2 * np.pi * trans["freq"] / medium["speed"] # Wave Number
    angularF = trans["radius"]/trans["focus"] # Angular Frequency
    abs_Coeff = medium["absCoeff"] * (pow((trans["freq"]/(1e6)),2)) # POWER LAW
    # Set Axes
    axial_min = 0.001
    axial_max = 2*d
    radial_min = -trans["radius"]
    dz = (axial_max - axial_min) / field["numAxialStep"]
    numZ = int(np.round((axial_max - axial_min)/dz)+1)
    dr = -1 * radial_min / (field["numRadialStep"]/2)
    numR = int(np.round((0 - radial_min)/dr)+1)
    # Theta Component
    thetaMax = np.arcsin(angularF)
    numT = 100
    dtheta = thetaMax/numT

    # Preallocation
    z_values = list()
    r_values = list()
    pressure_field = np.empty((numZ,numR), dtype=float)
    # Rayleigh Integral
    z = axial_min
    for zz in range(0,numZ):
        r = radial_min
        for rr in range(0,numR):
            p = 0.5 * np.exp(1j * k * np.sqrt(z*z + r*r)) / np.sqrt(z*z + r*r) * dtheta
            for tt in range(1,numT+1):
                theta = tt * dtheta
                numP = (2*tt+1)
                dphi = (2*np.pi)/numP
                e1 = 0
                for pp in range(0,numP):
                    phi = dphi * pp
                    rf = np.sqrt((pow((d*np.sin(theta)),2))+r*r-2*np.sin(theta)*np.absolute(r)*d*np.cos(phi)+pow((z-d+d*np.cos(theta)),2))
                    e1 = e1 + np.exp(1j*k*rf)/rf
                p = p + e1*np.sin(theta)/(2*tt+1)
            amplitude = abs(p)*k*d*d*dtheta*np.exp(-abs_Coeff*(z-axial_min))
            pressure_field[zz,rr] = amplitude
            if zz == 0: 
                r_values.append(r)
            r = r + dr
        z_values.append(z)
        z = z + dz
        print((np.round(100 * (zz+1)/numZ)),'%')
    
    # Set Bottom Halves
    r_value_bothalf = r_values
    pressure_field_bothalf = pressure_field
    # Radial Symmetry (reflect pressure values across z-axis)
    pressure_field_tophalf = np.fliplr(pressure_field_bothalf)
    pressure_field = np.rot90(np.hstack((pressure_field_bothalf,pressure_field_tophalf[:,1:])),1)
    r_value_tophalf = np.flipud(np.absolute(r_value_bothalf))
    r_values = np.concatenate((r_value_bothalf,r_value_tophalf[1:]))
    # Fix Dividing by 0 at the first center point
    r_CenterIdx = round((-1*radial_min/dr))
    r_values[r_CenterIdx] = 1e-20
    pressure_field[r_CenterIdx,0] = 0

    # Finish
    print('Rayleigh Integral Complete')
    iscomplete = 1
    return pressure_field, z_values, r_values, iscomplete

In [ ]:
trans = dict(freq = 1*1e6,radius = 0.02, focus = 0.05, initPressure = 1*1e6)
medium = dict(name='Water', speed=1500, density=1000, absCoeff=0.025, specHeatCap=4180, thermDiff=1.46*1e-7)
field = dict(numAxialStep = 100, numRadialStep = 100)
# Rayleigh Integral
df_pressure2D, z_axis, r_axis, iscomplete = generateField(trans,medium,field)

1.0 %
2.0 %
3.0 %
4.0 %
5.0 %
6.0 %
7.0 %
8.0 %
9.0 %
10.0 %
11.0 %
12.0 %
13.0 %
14.0 %
15.0 %
16.0 %
17.0 %
18.0 %
19.0 %
20.0 %
21.0 %
22.0 %
23.0 %
24.0 %
25.0 %
26.0 %
27.0 %
28.0 %
29.0 %
30.0 %
31.0 %
32.0 %
33.0 %
34.0 %
35.0 %
36.0 %
37.0 %
38.0 %
39.0 %
40.0 %
41.0 %
42.0 %
43.0 %
44.0 %
45.0 %
46.0 %
47.0 %
48.0 %
49.0 %
50.0 %
50.0 %
51.0 %
52.0 %
53.0 %
54.0 %
55.0 %
56.0 %
57.0 %
58.0 %
59.0 %
60.0 %
61.0 %
62.0 %
63.0 %
64.0 %
65.0 %
66.0 %
67.0 %
68.0 %
69.0 %
70.0 %
71.0 %
72.0 %
73.0 %
74.0 %
75.0 %
76.0 %
77.0 %
78.0 %
79.0 %
80.0 %
81.0 %
82.0 %
83.0 %
84.0 %
85.0 %
86.0 %
87.0 %
88.0 %
89.0 %
90.0 %
91.0 %
92.0 %
93.0 %
94.0 %
95.0 %
96.0 %
97.0 %
98.0 %
99.0 %
Rayleigh Integral Complete


In [ ]:
# Check and Return Filenames
if iscomplete == 0:
    # Error
    filename1="ERROR_df"
    filename2="ERROR_z"
    filename3="ERROR_r"
else:
    print('Yay!')

In [ ]:
# Convert Data to Pandas Data Frame
df_pressure2D = pd.DataFrame(df_pressure2D)
z_axis = pd.DataFrame(z_axis)
r_axis = pd.DataFrame(r_axis)
# File Names and Save to CSV
filename1="df_pressure2D_test.csv"
df_pressure2D.to_csv(filename1)
filename2="z_axis_test.csv"
z_axis.to_csv(filename2)
filename3="r_axis_test.csv"
r_axis.to_csv(filename3)
# Return Links
directory = os.getcwd()  
filename1_path = directory + "\\" +  filename1
filename2_path = directory + "\\" +  filename2
filename3_path = directory + "\\" +  filename3

print(filename1_path)
print(filename2_path)
print(filename3_path)

NameError: name 'pd' is not defined